# Assignment 4 for Course 1MS041
Make         sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline         and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `mammography.mat` that you can download from [http://odds.cs.stonybrook.edu/mammography-dataset/](http://odds.cs.stonybrook.edu/mammography-dataset/). Below you can find the code to load this file into `X` and `Y`, you just need to put the file in your `data` folder. During mammography the doctor would be looking for something called `calcification`, which is calcium deposits in the breast tissue and is used as an indicator of cancer. In this dataset the `X` corresponds to some measurements, there are 6 features. The `Y` is a 0-1 label where 1 represents calcification and 0 does not.

1. [3p] Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, I have prepared a shuffling with a fixed seed, this way I can make sure that we all did the same splits. That is [train,test,validation] is the splitting layout.
2. [4p] Train a machine learning model on the training data (you are free to choose it yourself). Hint: you could always try `LogisticRegression`, but for it to work well you would need `StandardScaler` and put everything in a `Pipeline`.
3. [3p] Use the classification report from `Utils` and compute the intervals for precision-recall etc on the test set with `union_bound correction` with 95% confidence.
4. [3p] You are interested in minimizing the average cost of your classifier, but first we must define it:
    * If someone is calcified but classified as not, we say it costs 30 **(this is the worst scenario)**
    * If someone is not calcified but classified as calcified, we say it costs 5 **(this probably only costs extra investigation)**
    * If someone is calcified but classified as calcified, costs 0 **(We did the right thing, no cost)**
    * If someone is not calcified but classified as not, costs 0 **(We did the right thing, no cost)**.

complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). What would be the cost of having a model that always classifies someone as not calcified on the test set?

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, we do that by checking say 100 evenly spaced proposal thresholds between 0 and 1, and use our testing data to compute the cost.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 95% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted, if we define the random variable
$$
    C = 30(1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
$$
then $C$ denotes the cost of a randomly chosen patient. In the above we are estimating $\mathbb{E}[C]$ using the empirical mean. However, since the number of calcifications in the population is fairly small and our classifier probably has a fairly high precision for the $0$ class, then $C$ should have fairly small variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [1]:
import scipy.io as so
import numpy as np
data = so.loadmat('data/mammography.mat')

np.random.seed(0)
shuffle_index = np.arange(0,len(data['X']))
np.random.shuffle(shuffle_index)

X = data['X'][shuffle_index,:]
Y = data['y'][shuffle_index,:].flatten()

In [2]:

# Part 1

# Split the X,Y into three parts, make sure that the sizes are
# (6709, 6), (1677, 6), (2797, 6), (6709,), (1677,), (2797,)

X_train, X_test, X_valid, Y_train, Y_test, Y_valid = X[:6709], X[6709:6709+1677], X[6709+1677:6709+1677+2797], Y[:6709], Y[6709:6709+1677], Y[6709+1677:6709+1677+2797]

# print(X_train.shape)
# print(X_test.shape)
# print(X_valid.shape)
# print(Y_train.shape)
# print(Y_test.shape)
# print(Y_valid.shape)

In [3]:

# Part 2
# Use X_train,Y_train to train a model from sklearn. Make sure it has the predict_proba function
# for instance LogisticRegression

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

model = Pipeline([
    ('scaler', StandardScaler()),  # Standardize features
    ('classifier', LogisticRegression())  # Logistic Regression model
])

model.fit(X_train, Y_train)
predictions = model.predict(X_test)


In [4]:
# from Utils import classification_report_interval
# import inspect 

# try:
#     print(inspect.getsource(classification_report_interval))
# except OSError:
#     print("Source not available")

# Part 3

# Compute precision and recall on the test set using
from Utils import classification_report_interval

# Each precision and recall should be a tuple, for instance you can write
# precision0 = (0.9,0.95)

#print(classification_report_interval(Y_test, predictions, alpha = 0.05, union_bound_correction=True))

precision0 = (0.94,1.00)
recall0 = (0.96,1.00)
precision1 = (0.33,1.00)
recall1 = (0.09,0.62)

# The code below will check that you supply the proper type
assert(type(precision0) == tuple)
assert(len(precision0) == 2)
assert(type(recall0) == tuple)
assert(len(recall0) == 2)
assert(type(precision1) == tuple)
assert(len(precision1) == 2)
assert(type(recall1) == tuple)
assert(len(recall1) == 2)

In [5]:

# Part 4

def cost(model,threshold,X,Y):
    pred_proba = model.predict_proba(X)[:,1]
    predictions = (pred_proba >= threshold)*1

    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost (cost per person)

     # Calculate the cost components
    false_positive_cost = 5  # Cost when someone is not calcified but classified as calcified
    false_negative_cost = 30  # Cost when someone is calcified but classified as not calcified

    # Calculate the cost for each individual
    individual_costs = (
        Y * (predictions == 0) * false_negative_cost +  # Calcified but classified as not
        (1 - Y) * (predictions == 1) * false_positive_cost  # Not calcified but classified as calcified
    )

    # Calculate the total cost and average cost per person
    total_cost = individual_costs.sum()
    average_cost_per_person = total_cost / len(Y)

    return average_cost_per_person

In [6]:

# Part 4

# Fill in the naive cost of the model that would classify all as non-calcified on the test set

naive_cost = cost(model, 0.5, X_test, np.zeros(1677))


In [7]:

# Part 5

thresholds = np.linspace(0, 1, 100)
costs = []

for threshold in thresholds:
    average_cost = cost(model, threshold, X_test, Y_test)
    costs.append(average_cost)

optimal_threshold = thresholds[costs.index(min(costs))]
cost_at_optimal_threshold = min(costs)

In [8]:

# Part 6

cost_at_optimal_threshold_validation = cost(model, optimal_threshold, X_valid, Y_valid)
# Report the cost interval as a tuple cost_interval = (a,b)

confidence_level = 0.95
num_samples = len(Y_valid)
epsilon = np.sqrt((1 / (2 * num_samples)) * np.log(2 / (1 - confidence_level)))

# Calculate the confidence interval
lower_bound = cost_at_optimal_threshold_validation - epsilon
upper_bound = cost_at_optimal_threshold_validation + epsilon
cost_interval = (lower_bound, upper_bound)

# The code below will tell you if you filled in the intervals correctly
assert(type(cost_interval) == tuple)
assert(len(cost_interval) == 2)

In [9]:

# Part 7

# Compute the cost variable C for each patient in the validation set
pred_proba_validation = model.predict_proba(X_valid)[:, 1]
predictions_validation = (pred_proba_validation >= optimal_threshold).astype(int)
cost_variable = 30 * (1 - predictions_validation) * Y_valid + 5 * (1 - Y_valid) * predictions_validation

# Compute the empirical mean and variance of the cost variable C
empirical_mean_C = np.mean(cost_variable)
empirical_variance_C = np.var(cost_variable, ddof=1)  # Use ddof=1 for sample variance

variance_of_C = np.var(cost_variable, ddof=1)

In [10]:

# Part 7

# A 95% confidence interval of the random variable C using Bennett's inequality

confidence_level = 0.95
num_samples = len(Y_valid)

# Calculate the confidence interval using Bennett's inequality
epsilon = np.sqrt(empirical_variance_C / (num_samples * confidence_level))
lower_bound_bennett = empirical_mean_C - epsilon
upper_bound_bennett = empirical_mean_C + epsilon

interval_of_C = (lower_bound_bennett, upper_bound_bennett)

assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

Numerical error 1.0408340855860843e-17
Numerical error 1.0408340855860843e-17
Beginning tests for problem 1

---------------------------------
Beginning test for part1
---------------------------------

-----Beginning test------
Shape of X_train correct it is (6709,6)
-----Ending test---------

-----Beginning test------
Shape of X_test correct it is (1677,6)
-----Ending test---------

-----Beginning test------
Shape of X_valid correct it is (2797,6)
-----Ending test---------

-----Beginning test------
Shape of Y_train correct it is (6709,)
-----Ending test---------

-----Beginning test------
Shape of Y_test correct it is (1677,)
-----Ending test---------

-----Beginning test------
Shape of Y_valid correct it is (2797,)
-----Ending test---------

---------------------------------
Beginning test for part2
---------------------------------

-----Beginning test------
Your model does produce the correct output shape when predict_proba is called on X_train
-----Ending test---------

-----Beg